# Multithreading: threads
_COSC 208, Introduction to Computer Systems, 2023-04-26_

## Announcements
* Project 5 due Thursday, May 4

## Outline
* Warm-up
* Threads and memory
* Pthreads API
* Creating multiple threads
* Returning values from threads
* Passing multiple parameters to threads

## Warm-up

_Consider the following set of processes:_

| Process | Arrival Time | Duration |
|---------|--------------|----------|
| A       | 0            | 30       |
| B       | 0            | 25       |
| C       | 5            | 15       |
| D       | 10           | 5        |

* Q1: _Draw the schedule when a Round Robin (RR) scheduling algorithm is used with a time quantum of 15._

```
  A              B              C              D    A              B       
|--------------|--------------|--------------|----|--------------|---------|
0              15             30             45   50             65        75
```

<p style="height:6em;"></p>

* Q2: _Compute the turnaround and wait time for each process based on the above schedule._

| Process | Turnaround | Wait |
|---------|------------|------|
| A       | 65         | 35   |
| B       | 75         | 50   |
| C       | 40         | 25   |
| D       | 40         | 35   |

<p style="height:10em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Threads and memory

* Threads are multiple execution contexts within the **same process**
    * Processes are multiple execution contexts within the **same machine**
* Because threads are within the same process, they share all of the process’s resources — memory, CPU time, file descriptors (i.e., open files), etc.
* Consequently, two threads can update the same variable

* Example

In [ ]:
#include <stdio.h>
#include <stdlib.h>
void *thread1_main(void *arg) {
    int *x = (int *)arg;
    *x += 1;
    return NULL;
}
void *thread2_main(void *arg) {
    int *y = (int *)arg;
    *y += 2;
    return NULL;
}
int main() {
    int *z = malloc(sizeof(int));
    *z = 0;
    // Create thread running thread1_main(z)
    // Create thread running thread2_main(z)
    // Wait for threads to finish
    printf("z is %d\n", *z);
}

```
z is 3
```

* Two processes cannot update the same variable — memory is not shared; must use inter-process communication mechanism to share information

* Q3: _What are all possible outputs produced by this program?_

In [ ]:
#include <stdio.h>
#include <stdlib.h>
void *thread_main(void *arg) {
    char *id = (char *)arg;
    printf("I am thread %c\n", *id);
    return NULL;
}
int main() {
    char *a = malloc(sizeof(char));
    *a = 'A';
    char *b = malloc(sizeof(char));
    *b = 'B';
    // Create thread running thread_main(a)
    // Create thread running thread_main(b)
    // Wait for threads to finish
}

```
I am thread A
I am thread B
```
OR
```
I am thread B
I am thread A
```

<p style="height:8em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Pthreads API

* Can create and wait for threads to finish, just like processes, but API is different
* Use the pthreads library—`#include <pthread.h>`
* `int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void*), void * arg)`
    * `thread`—a struct that stores metadata for the thread
    * `attr`—configuration settings for the thread
    * `start_routine`—the function to start executing when the thread starts
        * Pass a pointer to a function
    * `arg`—an argument passed to the aforementioned function
    * _How do we create a new process?_—`fork`
* `int pthread_join(pthread_t thread, void **value_ptr)`
    * `thread`—the same struct passed at thread creation; used to identify the thread we want to wait for
    * `value_ptr`—the location where the function return value should be stored
        * Notice it’s a pointer to a void pointer and the `start_routine` function specified in create returns a void pointer
    * _How do we wait for a process to finish?_ — `wait` or `waitpid`

* Q4: _What are all possible outputs produced by this program?_

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
void *increment(void *arg) {
    int *num = (int*)arg;
    *num += 1;
    return NULL;
}
void *zero(void *arg) {
    int *num = (int*)arg;
    *num = 0;
    return NULL;
}
int main() {
    pthread_t thread1, thread2;
    int *i = malloc(sizeof(int));
    *i = 5;
    pthread_create(&thread1, NULL, &increment, i);
    pthread_create(&thread2, NULL, &zero, i);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
    printf("i=%d\n", *i);
}

```
i=0
```
OR
```
i=1
```

<p style="height:5em;"></p>

* Q5: _What are all possible outputs produced by this program?_

In [1]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
void *printer(void *arg) {
    char *ch = (char*)arg;
    printf("I am %c\n", *ch);
    return NULL;
}
int main() {
    pthread_t thread1, thread2;
    char *ch = malloc(sizeof(char));
    *ch = 'P';
    pthread_create(&thread1, NULL, &printer, ch);
    pthread_join(thread1, NULL);
    *ch = 'Q';
    pthread_create(&thread2, NULL, &printer, ch);
    pthread_join(thread2, NULL);
}

I am P
I am Q


<div style="page-break-after:always;"></div>

## Extra practice

* Q6: _What are all possible outputs produced by this program?_

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
void *printer(void *arg) {
    char *ch = (char*)arg;
    printf("I am %c\n", *ch);
    return NULL;
}
int main() {
    pthread_t thread1, thread2;
    char *ch = malloc(sizeof(char));
    *ch = 'A';
    pthread_create(&thread1, NULL, &printer, ch);
    *ch = 'B';
    pthread_create(&thread2, NULL, &printer, ch);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
}

```
I am A
I am B
```
OR
```
I am B
I am B
```

<p style="height:8em;"></p>

* Q7: _What are all possible outputs produced by this program?_

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
void *printer2(void *arg) {
    char *ch = (char*)arg;
    printf("Start %c\n", *ch);
    printf("End %c\n", *ch);
    return NULL;
}
int main() {
    pthread_t thread1, thread2;
    char *ch1 = malloc(sizeof(char));
    *ch1 = 'X';
    char *ch2 = malloc(sizeof(char));
    *ch2 = 'Y';
    pthread_create(&thread1, NULL, &printer2, ch1);
    pthread_create(&thread2, NULL, &printer2, ch2);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
}

```
Start X
End X
Start Y
End Y
```
OR
```
Start Y
End Y
Start X
End X
```
OR
```
Start X
Start Y
End Y
End X
```
OR
```
Start X
Start Y
End X
End Y
```
OR
```
Start Y
Start X
End X
End Y
```
OR
```
Start Y
Start X
End Y
End X
```